# Description

<b>Stores

Anonymized information about the 45 stores, indicating the type and size of store

<b>Features

Contains additional data related to the store, department, and regional activity for the given dates.

Store - the store number

Date - the week

Temperature - average temperature in the region

Fuel_Price - cost of fuel in the region

MarkDown1-5 - anonymized data related to promotional markdowns. MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA

CPI - the consumer price index

Unemployment - the unemployment rate

IsHoliday - whether the week is a special holiday week

<b>Sales

Historical sales data, which covers to 2010-02-05 to 2012-11-01. Within this tab you will find the following fields:


Store - the store number

Dept - the department number

Date - the week

Weekly_Sales -  sales for the given department in the given store

IsHoliday - whether the week is a special holiday week

# Initializing

In [3]:
pip install -U pydantic

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [5]:
#importing libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('rainbow')
sns.color_palette("Paired")
#sns.set_context('notebook',font_scale=1.5, rc={'lines.linewidth':2.5})
sns.set_context('talk')
from pandas_profiling import ProfileReport

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.6/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.6/u/import-error

In [ ]:
#loading files
try:
    features = pd.read_csv("/datasets/features.csv")

except FileNotFoundError:
    features = pd.read_csv("features.csv")

In [ ]:
try:
    sales = pd.read_csv("/datasets/sales.csv")

except FileNotFoundError:
    sales = pd.read_csv("sales.csv")

In [ ]:
try:
    stores = pd.read_csv("/datasets/stores.csv")

except FileNotFoundError:
    stores = pd.read_csv("stores.csv")

# Modyfing Data

## standaryzing data

In [ ]:
features.head()

In [ ]:
features.info()

In [ ]:
#lower casing columns 
features.columns = features.columns.str.lower()

In [ ]:
#changing data type to datetime
features['date'] = pd.to_datetime(features['date'], format='%Y/%m/%d', infer_datetime_format=True)

In [ ]:
features.info()

In [ ]:
sales.head()

In [ ]:
sales.info()

In [ ]:
#lower casing columns names
sales.columns = sales.columns.str.lower()

In [ ]:
#changing type of data to datetime
sales['date'] = pd.to_datetime(sales['date'], format='%Y/%m/%d', infer_datetime_format=True)

In [ ]:
stores.head()

In [ ]:
stores.info()

In [ ]:
#lower casing columns names
stores.columns = stores.columns.str.lower()

## cleaning data

In [ ]:
features.head()

In [ ]:
#markdowns columns name
markdowns = ['markdown1','markdown2','markdown3','markdown4','markdown5']

#filling missing values in markdown
features[markdowns].fillna(0)

In [ ]:
features.info()

## enhancing data

In [ ]:
sales.head()

In [ ]:
sales.info()

In [ ]:
#adding month column
sales['month'] = sales['date'].dt.month

In [ ]:
#adding year column
sales['year'] = sales['date'].dt.year

In [ ]:
features.sort_values(by='date',ascending=True).head()

In [ ]:
#add temperature and fuel price to table
sales_temp_fuel = sales.merge(features[['date','store','temperature','fuel_price']], on=('date','store'), how='left')

In [ ]:
#merging sales and stores
sales_stores  = stores.merge(sales_temp_fuel, on='store')
sales_stores.head()

In [ ]:
sales_stores.info()

In [ ]:
#creating temperature categories
temp_category = pd.cut(sales_stores['temperature'], bins=5, labels=['very cold', 'cold', 'warm','hot','very hot'])
temp_category

In [ ]:
#adding temperature categories
sales_stores['temp_category'] = temp_category

In [ ]:
sales_stores.head()

# Analysis / Insights

### best and worst 10, sales 2011

In [ ]:
sales_stores.info()

In [ ]:
sales_stores.head()

In [ ]:
#filtering by 2011 year
sales_stores_2011 = sales_stores.query('date > ="2011-01-01" and date <= "2011-12-31"')

In [ ]:
sales_stores_2011

In [ ]:
#grouping sales by stores
total_sales_2011 = sales_stores_2011.groupby('store')['weekly_sales'].sum().reset_index(name='total')
total_sales_2011.head()

In [ ]:
#plotting total sales from 2011 by store
plt.figure(figsize=(20,8))
sns.barplot(data=total_sales_2011, x='store', y='total')

plt.show()

In [ ]:
#top 10 store sales 2011
top10_2011 = total_sales_2011.sort_values(by='total',ascending=False).head(10)
top10_2011

In [ ]:
#top 10 stores in 2011
plt.figure(figsize=(12,8))
sns.barplot(data=top10_2011, x='store', y='total')
plt.show()

In [ ]:
bottom10_2011 = total_sales_2011.sort_values(by='total',ascending=False).tail(10)
bottom10_2011

In [ ]:
#bottom 10 stores in 2011
plt.figure(figsize=(12,8))
sns.barplot(data=bottom10_2011, x='store', y='total')
plt.show()

### year tracking

In [ ]:
sales_stores_2011.head()

In [ ]:
sales_stores_2011['month'].unique()

In [ ]:
#grouping by monthly store revenue in 2011
monthly_sales_stores = sales_stores_2011.groupby(['store','month']).agg(total=('weekly_sales','sum')).reset_index()
monthly_sales_stores.head()

In [ ]:
#plotting monthly sales of all stores
plt.figure(figsize=(12,8))
sns.lineplot(data=monthly_sales_stores, x='month', y='total',hue='store')
plt.show()

In [ ]:
top10 = top10_2011['store'].tolist()
top10

In [ ]:
#plotting monthly sales of top 10 stores
plt.figure(figsize=(12,8))
sns.lineplot(data=monthly_sales_stores.query('store == @top10'), x='month', y='total',hue='store')
plt.show()

In [ ]:
month_sales = sales_stores_2011.groupby('month')['weekly_sales'].mean().reset_index(name='avg_sales').round(2)
month_sales.head()

In [ ]:
#plot sales per month 2011 
plt.figure(figsize=(12,8))
sns.lineplot(data=month_sales, x='month', y='avg_sales')
plt.show()

## when is holidays

In [ ]:
sales_stores.head()

In [ ]:
holiday_week_sales = sales_stores.query('isholiday == True')
holiday_week_sales

In [ ]:
non_holiday_week_sales = sales_stores.query('isholiday == False')
non_holiday_week_sales

In [ ]:
total_hols = holiday_week_sales.groupby('store')['weekly_sales'].sum().reset_index(name='total').sort_values(by='total',ascending=False)
total_hols.head()

In [ ]:
top10_hols = total_hols['store'].head(10).tolist()
top10_hols

In [ ]:
#plot top 10 stores performance on holidays week
plt.figure(figsize=(12,8))
sns.barplot(data=total_hols.query('store == @top10_hols'), x='store', y='total')
plt.show()

In [ ]:
bottom10_hols = total_hols['store'].tail(10).tolist()
bottom10_hols

In [ ]:
#plot worst 10 performance stores on holidays
plt.figure(figsize=(12,8))
sns.barplot(data=total_hols.query('store == @bottom10_hols'), x='store', y='total')
plt.show()

In [ ]:
total_non_hols = non_holiday_week_sales.groupby('store')['weekly_sales'].sum().reset_index(name='total').sort_values(by='total',ascending=False)
total_non_hols.head()

In [ ]:
top10_non_hols = total_non_hols['store'].head(10).tolist()
top10_non_hols

In [ ]:
#plot top 10 best performance on non holidays 
plt.figure(figsize=(12,8))
sns.barplot(data=total_non_hols.query('store == @top10_non_hols'), x='store', y='total')
plt.show()

In [ ]:
bottom10_non_hols = total_non_hols['store'].tail(10).tolist()
bottom10_non_hols

In [ ]:
#plot top 10 worst performance on non holidays
plt.figure(figsize=(12,8))
sns.barplot(data=total_non_hols.query('store == @bottom10_non_hols'), x='store', y='total')
plt.show()

## sales on temperature

In [ ]:
sales_stores_2011.head()

In [ ]:
#grouping category by total sales
temp_sales = sales_stores_2011.groupby('temp_category')['weekly_sales'].sum().reset_index(name='total_sales').sort_values(by='total_sales',ascending=False)
temp_sales

In [ ]:
#plot sales by temp category
plt.figure(figsize=(12,8))
sns.barplot(data=temp_sales, x='temp_category',y='total_sales')
plt.show()

In [ ]:
#grouping month by temperature average
month_temp = sales_stores_2011.groupby('month')['temperature'].mean().reset_index(name='avg_temp')
month_temp.head()

In [ ]:
#plot month by temperature
plt.figure(figsize=(12,8))
sns.lineplot(data=month_temp, x='month', y='temperature')
plt.show()